
- input data
    1. raw data 1sec window / 1hop, 2hop, 3hop concatenate
    2. raw data 5sec window / 1hop, 2hop, 3hop concatenate
    3. hidden feature from 파악 task 5sec window RNN model / 1hop, 2hop, 3hop, concatenate
    
- model
    1. FNN
    2. LSTM
    3. LSTM + 0hop

simulation map
<img src="./map.png" width=500/>

sensorData_x.txt is made from region x in above map

# Prepare data for prediction task

In [1]:
# import modules
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.backend import tensorflow_backend as K
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation
from keras.utils import plot_model, to_categorical
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.initializers import he_normal
from keras.utils.training_utils import multi_gpu_model

from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm

Using TensorFlow backend.


In [2]:
# return current graph topology
def get_adj_matrix():
    arr = [[0]*8 for x in range(8)]
    for i in range(8):
        before, after = (i-1)%8, (i+1)%8
        arr[i][before] = 1
        arr[i][after] = 1
        
    return arr

# return distance matrix
# using Floyd-Warshall
def get_distance_matrix(adj_matrix):
    result = [row[:] for row in adj_matrix]
    N = len(result)
    
    for i in range(N):
        for j in range(N):
            if i != j and result[i][j] == 0:
                result[i][j] = 8
    
    for k in range(N):
        for i in range(N):
            for j in range(N):
                result[i][j] = min(result[i][j], result[i][k] + result[k][j])
                    
    return result




In [3]:
adj_matrix = get_adj_matrix()
dist_matrix = get_distance_matrix(adj_matrix)
num_nodes = len(adj_matrix)

print('adj matrix')
for row in adj_matrix:
    print(row)
    
print('dist matrix')
for row in dist_matrix:
    print(row)
    
print('num_nodes:', num_nodes)



adj matrix
[0, 1, 0, 0, 0, 0, 0, 1]
[1, 0, 1, 0, 0, 0, 0, 0]
[0, 1, 0, 1, 0, 0, 0, 0]
[0, 0, 1, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 0, 1, 0, 0]
[0, 0, 0, 0, 1, 0, 1, 0]
[0, 0, 0, 0, 0, 1, 0, 1]
[1, 0, 0, 0, 0, 0, 1, 0]
dist matrix
[0, 1, 2, 3, 4, 3, 2, 1]
[1, 0, 1, 2, 3, 4, 3, 2]
[2, 1, 0, 1, 2, 3, 4, 3]
[3, 2, 1, 0, 1, 2, 3, 4]
[4, 3, 2, 1, 0, 1, 2, 3]
[3, 4, 3, 2, 1, 0, 1, 2]
[2, 3, 4, 3, 2, 1, 0, 1]
[1, 2, 3, 4, 3, 2, 1, 0]
num_nodes: 8


In [4]:
# load txt data and apply label
def load_data():
    # load data
    data = []
    for i in tqdm(range(num_nodes)):
        curr = np.loadtxt('./data/123456people_320000frame_simple/sensorData_{}.txt'.format(i+1), delimiter=',', dtype=np.int32)
        print('i:', i, 'shape:', curr.shape)
        data.append(curr)

    # data.shape: (time, node, 9)    
    data = np.stack(data, axis=1)
    
    # apply label
    bins=[1, 4, 7]
    data[:, :, 0] = np.digitize(data[:, :, 0], bins)
    
    return data


data = load_data()
print(data.shape) # (time, node, 9)

i: 0 shape: (320000, 9)
i: 1 shape: (320000, 9)
i: 2 shape: (320000, 9)
i: 3 shape: (320000, 9)
i: 4 shape: (320000, 9)
i: 5 shape: (320000, 9)
i: 6 shape: (320000, 9)
i: 7 shape: (320000, 9)

(320000, 8, 9)


In [5]:
# dist_matrix: NxN distnace matrix
# src: node index, not node number, 0<=src<=7, e.g. in case of 'S1', src=0
# hop: distance from source node
# return: node indexes whose distance from src index equals to hop
def get_nhop_index(dist_matrix, src, hop):
    result = []
    for i, dist in enumerate(dist_matrix[src]):
        if dist == hop:
            result.append(i)
            
    return result

In [6]:
# return x, y data for using at train time
# at each unit frame, we can make N(number of nodes) data
# beacuse our network takes [1hop avg, 2hop, avg, 3hop avg] as input and [label] as output

# data: numpy nd array, shape: (time, node, 9)
# num_frame: how many frames in window
# after: time difference between end of window and expectation
# ---------               |
# |       |               |
# ---------               |
#
# |       |               |
# t    t+frame_size-1   t+frame_size+after-1
#
# return (x, y), x.shape = (N, frame_size, 8*num_hops), y.shape = (N,)
def get_frame_xy(data, frame_size, after):
    x, y = [], []
    num_times, num_nodes, _ = data.shape
    for t in tqdm(range(num_times-frame_size-after)):
        for n in range(num_nodes):
            # be careful to calculate index
            label = data[t+frame_size+after-1, n, 0]
            avgs = []
            for hop in range(1, 4):
                idx = get_nhop_index(dist_matrix, src=n, hop=hop)
                # node-wise mean, shape: (frame_size, 8)
                avgs.append(np.mean(data[t:t+frame_size, idx, 1:], axis=1)) 

            x.append(np.concatenate(avgs, axis=1))
            y.append(label)
    
    x = np.array(x) # shape: (N, frame_size, 8*3), beacuse we use 3 kinds of avg hops
    y = np.array(y) # shape: (N,)
    
    # balance between labels
    labels, counts = np.unique(y, return_counts=True)
    min_count = np.min(counts)
    
    # use at least 2500 examples if available(e.g. label=0,1,2)
    min_count = max(2500, min_count) 

    # shuffle
    np.random.seed(777) # fix seed
    idx = np.random.permutation(x.shape[0])
    x, y = x[idx], y[idx]
    
    x = [x[y==label][:min_count, :, :] for label in labels]
    x = np.concatenate(x, axis=0)
    y = [y[y==label][:min_count] for label in labels]
    y = np.concatenate(y, axis=0)
    
    return x, y

# Model

In [53]:
class FNN():
    # input dim = frame_size*8*num_hops
    def __init__(self, input_dim):
        model = Sequential()
        init = he_normal()
        
        model.add(Dense(64, input_dim=input_dim, kernel_initializer=init))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(Dense(128, kernel_initializer=init))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(Dense(64, kernel_initializer=init))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        
        model.add(Dense(4, kernel_initializer=init)) # output layer
        model.add(Activation('softmax'))
        
        model = multi_gpu_model(model, gpus=4)
        optimizer = Adam(lr=0.0001)
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        self.model = model
        
    def fit(self, x, y, epochs, batch_size):
        early_stopping = EarlyStopping(monitor='val_acc', patience=20)
        return self.model.fit(x, y, epochs=epochs, batch_size=batch_size,
                             validation_split=0.25,
                             callbacks=[early_stopping])
        
    def evaluate(self, x, y, batch_size=256):
        return self.model.evaluate(x, y, batch_size=batch_size)

In [54]:
class LSTM_net():
    # input_dim = 8 * num_hops
    # timesteps = frame_size
    def __init__(self, timesteps, input_dim):
        model = Sequential()
        
        model.add(LSTM(128, return_sequences=True, input_shape=(timesteps, input_dim)))
        model.add(BatchNormalization())
        
        model.add(LSTM(128, return_sequences=True))
        model.add(BatchNormalization())
        
        model.add(LSTM(32))
        model.add(BatchNormalization())
        
        model.add(Dense(128, activation='relu'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        
        model.add(Dense(4))
        model.add(BatchNormalization())
        model.add(Activation('softmax'))

        model = multi_gpu_model(model, gpus=4)
        optimizer = Adam(lr=0.0003)
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        self.model = model
        
    def fit(self, x, y, epochs, batch_size):
        early_stopping = EarlyStopping(monitor='val_acc', patience=20)
        return self.model.fit(x, y, epochs=epochs, batch_size=batch_size, 
                              validation_split=0.25,
                              callbacks=[early_stopping])
        
    def evaluate(self, x, y, batch_size=256):
        return self.model.evaluate(x, y, batch_size=batch_size)

In [30]:
# # for custom layer
# from keras.engine.topology import Layer
# from keras.initializers import Orthogonal

# # to use custom layer implement 'build', 'call', 'compute_shape'
# class GCN(Layer):
#     def __init__(self, num_hiddens, A, D, **kwargs):
#         self.num_hiddens = num_hiddens
#         self.M = np.matmul(np.matmul(D, A), D)
#         super().__init__(**kwargs)

#     def build(self, inshape):
#         self.w = self.add_weight("w", (inshape[1], self.num_hiddens),
#                                 initializer=Orthogonal)
#         self.b = self.add_weight("b", (self.num_hiddens,))
#         super().build(inshape)

#     def call(self, x):
#         # return K.dot(x, self.w) + self.b
#         return K.dot(K.dot(self.M, x), self.w) + self.b

#     def compute_output_shape(self, inshape):
#         return (inshape[0], self.num_hiddens)



# class GCN_net():
#     def __init__(self, input_dim, adj_matrix):
#         A = np.array(adj_matrix) + np.eye(len(adj_matrix)) # A hat, (NxN)
#         D = np.diag(1 / np.sqrt(np.sum(A, axis=1))) # D hat -1/2, (NxN)
        
#         model = Sequential()
#         model.add(GCN(32, A, D))
#         model.add(Activation('relu'))
        
#         model.add(GCN(32, A, D))
#         model.add(Activation('relu'))
        
# #         model = multi_gpu_model(model, gpus=4)
#         optimizer = Adam()
#         model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
#         self.model = model
        
#     def fit(self, x, y, epochs, batch_size):
#         early_stopping = EarlyStopping(monitor='val_acc', patience=20)
#         return self.model.fit(x, y, epochs=epochs, batch_size=batch_size, 
#                               validation_split=0.25,
#                               callbacks=[early_stopping],
#                               verbose=1)

#     def evaluate(self, x, y, batch_size=256):
#         return self.model.evaluate(x, y, batch_size=batch_size)

# Train and evaluate model

In [8]:
# plot loss, accuracy graph and save it to file
def plot_history(history, modelname, path):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    
    # Plot training & validation accuracy values
    ax1.plot(history.history['acc'])
    ax1.plot(history.history['val_acc'])
    ax1.set_title('Model accuracy')
    ax1.set_ylabel('Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.legend(['Train', 'Valid'], loc='upper left')
    
    # Plot training & validation loss values
    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_title('Model loss')
    ax2.set_ylabel('Loss')
    ax2.set_xlabel('Epoch')
    ax2.legend(['Train', 'Valid'], loc='upper left')

    fig.savefig(os.path.join(path, modelname+'.png'))

In [10]:
import logging
from datetime import datetime
import os

dir_name = os.path.join('.', datetime.today().strftime('%Y-%m-%d-%H-%M'))
if not os.path.isdir(dir_name):
    os.mkdir(dir_name)

In [ ]:
# # def train_GCN():
# logger = logging.getLogger('GCN_LOGGER')
# logger.setLevel(logging.INFO)
# file_handler = logging.FileHandler(os.path.join(dir_name, 'GCN.log'))
# logger.addHandler(file_handler)

# logger.info("Train GCN start")

In [12]:
# num_hops = 3
# frame_size = 8
# after = 80

# x, y = get_frame_xy(data, frame_size, after)

# # (N, frame_size, 8*num_hops) -> (N, frame_size*8*num_hops)
# x = np.reshape(x, (-1, frame_size*8*num_hops))
# # one-hot encoding
# y = to_categorical(y, num_classes=4)

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,
#                                                     random_state=42,
#                                                     stratify=y)

In [32]:
# model = GCN_net(frame_size*8*num_hops, adj_matrix)
# # model.model.summary()

In [33]:
# history = model.fit(x_train, y_train, epochs=1000, batch_size=256)
# loss, accuracy = model.evaluate(x_test, y_test, batch_size=256)

TypeError: __call__() missing 1 required positional argument: 'shape'

In [25]:
# # test set accuracy
# logger.info('test accuracy: {}'.format(accuracy))

# # plot history
# plot_history(history, 'FNN_framesize{}_after{}'.format(frame_size, after), dir_name)

# train_GCN()

KeyboardInterrupt: 

In [ ]:
def train_FNN():
    logger = logging.getLogger('FNN_LOGGER')
    logger.setLevel(logging.INFO)
    file_handler = logging.FileHandler(os.path.join(dir_name, 'FNN.log'))
    logger.addHandler(file_handler)

    logger.info("Train FNN start")


    num_hops = 3
    for frame_size in [8, 40]:
        for after in [80, 160, 240]:
            logger.info('frame_size: {}, after: {}'.format(frame_size, after))
            x, y = get_frame_xy(data, frame_size, after)
            
            # (N, frame_size, 8*num_hops) -> (N, frame_size*8*num_hops)
            x = np.reshape(x, (-1, frame_size*8*num_hops))
            # one-hot encoding
            y = to_categorical(y, num_classes=4)
            
            model = FNN(frame_size*8*num_hops)
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,
                                                                random_state=42,
                                                                stratify=y)
                        
            history = model.fit(x_train, y_train, epochs=1000, batch_size=256)
            loss, accuracy = model.evaluate(x_test, y_test, batch_size=256)

            # test set accuracy
            logger.info('test accuracy: {}'.format(accuracy))

            # plot history
            plot_history(history, 'FNN_framesize{}_after{}'.format(frame_size, after), dir_name)


In [ ]:
def train_LSTM():
    logger = logging.getLogger('LSTM_LOGGER')
    logger.setLevel(logging.INFO)
    file_handler = logging.FileHandler(os.path.join(dir_name, 'LSTM.log'))
    logger.addHandler(file_handler)

    logger.info("Train LSTM start")


    num_hops = 3
    for frame_size in [8, 40]:
        for after in [80, 160, 240]:
            logger.info('frame_size: {}, after: {}'.format(frame_size, after))
            x, y = get_frame_xy(data, frame_size, after)
            
            # one-hot encoding
            y = to_categorical(y, num_classes=4)
            
            model = LSTM_net(timesteps=frame_size, input_dim=8*num_hops)
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,
                                                                random_state=42,
                                                                stratify=y)
                        
            history = model.fit(x_train, y_train, epochs=1000, batch_size=256)
            loss, accuracy = model.evaluate(x_test, y_test, batch_size=256)

            # test set accuracy
            logger.info('test accuracy: {}'.format(accuracy))

            # plot history
            plot_history(history, 'LSTM_framesize{}_after{}'.format(frame_size, after), dir_name)


In [ ]:
train_FNN()

In [ ]:
train_LSTM()